<a href="https://colab.research.google.com/github/weathon/kNN_Research/blob/main/Credit_Card_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp kaggle.json /root/.kaggle/
!kaggle datasets download -d sakshigoyal7/credit-card-customers
!unzip credit-card-customers.zip

credit-card-customers.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  credit-card-customers.zip
  inflating: BankChurners.csv        


In [ ]:
import pandas as pd
import numpy as np

### Data Washing

In [69]:
BankChurners = pd.read_csv("BankChurners.csv")

In [70]:
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1"]
del BankChurners["Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"]

In [71]:
Y = BankChurners["Attrition_Flag"]
del BankChurners["Attrition_Flag"]
del BankChurners["CLIENTNUM"]
X = BankChurners

In [72]:
Y = Y.replace("Existing Customer",0).replace("Attrited Customer",1)

In [74]:
X["Gender"] = X["Gender"].replace("M",1).replace("F",0)
X["Marital_Status"] = X["Marital_Status"].replace("Married",0).replace("Single",1).replace("Divorced",2).replace("Unknown",3)
X["Education_Level"] = X["Education_Level"].replace("Uneducated",0).replace("High School",1).replace("College",2).replace("Graduate",3).replace("Post-Graduate",4).replace("Doctorate",5).replace("Unknown",-1)

In [77]:
X.Card_Category.value_counts

0          Blue
1          Blue
2          Blue
3          Blue
4          Blue
          ...  
10122      Blue
10123      Blue
10124      Blue
10125      Blue
10126    Silver
Name: Card_Category, Length: 10127, dtype: object